In [1]:
import numpy as np
import tensorflow as tf
import math
import cv2
import matplotlib.pyplot as plt


In [2]:
arr=np.array([[[0,0,0],[0,0,0]],[[0,0,0],[0,0,0]]])
arr=arr.flatten()
arr[3:6]=1

print(len(arr))

12


In [3]:
def label_oneshot(label):
    new_label=[]
    for i in range(len(label)):
        arr=np.zeros((label[i][:,:,0].shape[0]*label[i][:,:,0].shape[1],38),np.uint64)
        flat_label=label[i][:,:,0].flatten()
        for x in range(len(flat_label)):
            arr[x,flat_label[x]]=1
        arr=np.reshape(arr,(label[i][:,:,0].shape[0],label[i][:,:,0].shape[1],38))
        new_label.append(arr)
    return new_label
def img_resize(img):
    for i in range(len(img)):
        img[i]=cv2.resize(img[i], None, fx=0.4, fy=0.4, interpolation=cv2.INTER_AREA)
    return img

In [4]:
def l_oneshot(label):
    arr=np.zeros((label[:,:,0].shape[0]*label[:,:,0].shape[1],38),np.uint64)
    flat_label=label[:,:,0].flatten()
    for x in range(len(flat_label)):
        arr[x,flat_label[x]]=1
    arr=np.reshape(arr,(label[:,:,0].shape[0],label[:,:,0].shape[1],38))
    return arr
def im_resize(img):
    img=cv2.resize(img, None, fx=0.4, fy=0.4, interpolation=cv2.INTER_AREA)
    return img

In [5]:
#train_label=[cv2.imread('data/train_labels/img13labels-'+str(i+1).zfill(6)+'.png') for i in range(800)]#5285

In [6]:
train_label=[l_oneshot(im_resize(cv2.imread('data/train_labels/img13labels-'+str(i+1).zfill(6)+'.png'))) for i in range(800)]#5285

In [7]:
train_image=[im_resize(cv2.imread('data/train_images/img-'+str(i+1).zfill(6)+'.jpg')) for i in range(800)]#5285

In [7]:
test_image=[im_resize(cv2.imread('data/test_images/img-'+str(i+1).zfill(6)+'.jpg')) for i in range(500)]#5050

In [ ]:
test_label=[l_oneshot(im_resize(cv2.imread('data/test_labels/img13labels-'+str(i+1).zfill(6)+'.png'))) for i in range(500)]#5050

In [8]:
#test_label=[cv2.imread('data/test_labels/img13labels-'+str(i+1).zfill(6)+'.png') for i in range(500)]#5050

In [64]:
# train_image=img_resize(train_image)
# train_label=img_resize(train_label)
# test_label=img_resize(test_label)
# test_image=img_resize(test_image)

In [8]:
print(train_label[0].shape)
print(len(train_label[0][:,:,0].flatten()))

(212, 292, 38)
61904


In [14]:
#new_test_label=label_oneshot(test_label)

In [15]:
print(test_label[0].shape)

(212, 292, 38)


In [ ]:
#new_train_label=label_oneshot(train_label)

In [12]:
print(len(train_label))

500


In [9]:
h=int(train_image[0].shape[0])
w=int(train_image[0].shape[1])
print(len(train_image))

800


In [10]:
print(train_image[0].shape)

(212, 292, 3)


In [79]:
#learning_rate=0.0001
training_epochs=100
batch_size = 8
tf.reset_default_graph()
x=tf.placeholder(tf.float32,[None,h,w,3])
y=tf.placeholder(tf.float32, [None,h,w,38])

In [80]:
print(x)


Tensor("Placeholder:0", shape=(?, 212, 292, 3), dtype=float32)


In [81]:
w1_1_1=tf.get_variable("w1_1_1", shape=[3,3,3,32], initializer=tf.contrib.layers.xavier_initializer_conv2d())
w1_1_2=tf.get_variable("w1_1_2", shape=[6,6,3,32], initializer=tf.contrib.layers.xavier_initializer_conv2d())
w1_2_1=tf.get_variable("w1_2_1", shape=[3,3,64,32], initializer=tf.contrib.layers.xavier_initializer_conv2d())
w1_2_2=tf.get_variable("w1_2_2", shape=[6,6,64,32], initializer=tf.contrib.layers.xavier_initializer_conv2d())

w2_1_1=tf.get_variable("w2_1_1", shape=[3,3,64,64], initializer=tf.contrib.layers.xavier_initializer_conv2d())
w2_1_2=tf.get_variable("w2_1_2", shape=[6,6,64,64], initializer=tf.contrib.layers.xavier_initializer_conv2d())
w2_2_1=tf.get_variable("w2_2_1", shape=[3,3,128,64], initializer=tf.contrib.layers.xavier_initializer_conv2d())
w2_2_2=tf.get_variable("w2_2_2", shape=[6,6,128,64], initializer=tf.contrib.layers.xavier_initializer_conv2d())

w3_1_1=tf.get_variable("w3_1_1", shape=[3,3,128,128], initializer=tf.contrib.layers.xavier_initializer_conv2d())
w3_1_2=tf.get_variable("w3_1_2", shape=[6,6,128,128], initializer=tf.contrib.layers.xavier_initializer_conv2d())
w3_2_1=tf.get_variable("w3_2_1", shape=[3,3,256,256], initializer=tf.contrib.layers.xavier_initializer_conv2d())
w3_2_2=tf.get_variable("w3_2_2", shape=[6,6,256,256], initializer=tf.contrib.layers.xavier_initializer_conv2d())
w3_3_1=tf.get_variable("w3_3_1", shape=[3,3,512,256], initializer=tf.contrib.layers.xavier_initializer_conv2d())
w3_3_2=tf.get_variable("w3_3_2", shape=[6,6,512,256], initializer=tf.contrib.layers.xavier_initializer_conv2d())

w3_4_1=tf.get_variable("w3_4_1", shape=[3,3,512,128], initializer=tf.contrib.layers.xavier_initializer_conv2d())
w3_4_2=tf.get_variable("w3_4_2", shape=[6,6,512,128], initializer=tf.contrib.layers.xavier_initializer_conv2d())
w3_4_3=tf.get_variable("w3_4_3", shape=[3,3,256,256], initializer=tf.contrib.layers.xavier_initializer_conv2d())


w4_1_1=tf.get_variable("w4_1_1", shape=[3,3,256,64], initializer=tf.contrib.layers.xavier_initializer_conv2d())
w4_1_2=tf.get_variable("w4_1_2", shape=[6,6,256,64], initializer=tf.contrib.layers.xavier_initializer_conv2d())
w4_2_1=tf.get_variable("w4_2_1", shape=[3,3,128,64], initializer=tf.contrib.layers.xavier_initializer_conv2d())
w4_2_2=tf.get_variable("w4_2_2", shape=[6,6,128,64], initializer=tf.contrib.layers.xavier_initializer_conv2d())

w5_1_1=tf.get_variable("w5_1_1", shape=[3,3,128,32], initializer=tf.contrib.layers.xavier_initializer_conv2d())
w5_1_2=tf.get_variable("w5_1_2", shape=[6,6,128,32], initializer=tf.contrib.layers.xavier_initializer_conv2d())
w5_2_1=tf.get_variable("w5_2_1", shape=[3,3,64,32], initializer=tf.contrib.layers.xavier_initializer_conv2d())
w5_2_2=tf.get_variable("w5_2_2", shape=[6,6,64,32], initializer=tf.contrib.layers.xavier_initializer_conv2d())

In [82]:
b1_1_1=tf.Variable(tf.random_normal([32]))
b1_1_2=tf.Variable(tf.random_normal([32]))
b1_2_1=tf.Variable(tf.random_normal([32]))
b1_2_2=tf.Variable(tf.random_normal([32]))

b2_1_1=tf.Variable(tf.random_normal([64]))
b2_1_2=tf.Variable(tf.random_normal([64]))
b2_2_1=tf.Variable(tf.random_normal([64]))
b2_2_2=tf.Variable(tf.random_normal([64]))

b3_1_1=tf.Variable(tf.random_normal([128]))
b3_1_2=tf.Variable(tf.random_normal([128]))
b3_2_1=tf.Variable(tf.random_normal([256]))
b3_2_2=tf.Variable(tf.random_normal([256]))
b3_3_1=tf.Variable(tf.random_normal([256]))
b3_3_2=tf.Variable(tf.random_normal([256]))
b3_4_1=tf.Variable(tf.random_normal([128]))
b3_4_2=tf.Variable(tf.random_normal([128]))


b4_1_1=tf.Variable(tf.random_normal([64]))
b4_1_2=tf.Variable(tf.random_normal([64]))
b4_2_1=tf.Variable(tf.random_normal([64]))
b4_2_2=tf.Variable(tf.random_normal([64]))

b5_1_1=tf.Variable(tf.random_normal([32]))
b5_1_2=tf.Variable(tf.random_normal([32]))
b5_2_1=tf.Variable(tf.random_normal([32]))
b5_2_2=tf.Variable(tf.random_normal([32]))

In [83]:
#############1 conv layer
l1_1_1=tf.nn.relu(tf.nn.conv2d(x,w1_1_1, strides=[1,1,1,1], padding= 'SAME')+b1_1_1)#(?,265,365,32)
l1_1_2=tf.nn.relu(tf.nn.conv2d(x,w1_1_2, strides=[1,1,1,1], padding= 'SAME')+b1_1_2)#(?,265,365,32)
l1_conv_1=tf.concat([l1_1_1, l1_1_2],3)#(?,265,365,64)
l1_2_1=tf.nn.relu(tf.nn.conv2d(l1_conv_1,w1_2_1, strides=[1,1,1,1], padding= 'SAME')+b1_2_1)#(?,265,365,32)
l1_2_2=tf.nn.relu(tf.nn.conv2d(l1_conv_1,w1_2_2, strides=[1,1,1,1], padding= 'SAME')+b1_2_2)#(?,265,365,32)
l1_conv_2=tf.concat([l1_2_1,l1_2_2],3)#(?,265,365,64)
l1=tf.nn.max_pool(l1_conv_2, ksize=[1,3,3,1], strides=[1,3,3,1], padding='SAME')#(?,89, 122,,64)

#############2 conv layer
l2_1_1=tf.nn.relu(tf.nn.conv2d(l1,w2_1_1, strides=[1,1,1,1], padding= 'SAME')+b2_1_1)#(?,89, 122,64)
l2_1_2=tf.nn.relu(tf.nn.conv2d(l1,w2_1_2, strides=[1,1,1,1], padding= 'SAME')+b2_1_2)#(?,89, 122,64)
l2_conv_1=tf.concat([l2_1_1, l2_1_2],3)#(?,88,121,128)
l2_2_1=tf.nn.relu(tf.nn.conv2d(l2_conv_1,w2_2_1, strides=[1,1,1,1], padding= 'SAME')+b2_2_1)#(?,89, 122,64)
l2_2_2=tf.nn.relu(tf.nn.conv2d(l2_conv_1,w2_2_2, strides=[1,1,1,1], padding= 'SAME')+b2_2_2)#(?,89, 122,64)
l2_conv_2=tf.concat([l2_2_1, l2_2_2],3)#(?,88,121,128)
l2=tf.nn.max_pool(l2_conv_2, ksize=[1,3,3,1], strides=[1,3,3,1], padding='SAME')#(?,30, 41,128)

#############3 conv layer
l3_1_1=tf.nn.relu(tf.nn.conv2d(l2,w3_1_1, strides=[1,1,1,1], padding= 'SAME')+b3_1_1)#(?,30, 41, 128)
l3_1_2=tf.nn.relu(tf.nn.conv2d(l2,w3_1_2, strides=[1,1,1,1], padding= 'SAME')+b3_1_2)#(?,30, 41, 128)
l3_conv_1=tf.concat([l3_1_1, l3_1_2],3)#(?,30, 41, 256)
l3_2_1=tf.nn.relu(tf.nn.conv2d(l3_conv_1,w3_2_1, strides=[1,1,1,1], padding= 'SAME')+b3_2_1)#(?,30, 41, 256)
l3_2_2=tf.nn.relu(tf.nn.conv2d(l3_conv_1,w3_2_2, strides=[1,1,1,1], padding= 'SAME')+b3_2_2)#(?,30, 41, 256)
l3_conv_2=tf.concat([l3_2_1, l3_2_2],3)#(?,30, 41, 512)
l3_3_1=tf.nn.relu(tf.nn.conv2d(l3_conv_2,w3_3_1, strides=[1,1,1,1], padding= 'SAME')+b3_3_1)#(?,30, 41, 256)
l3_3_2=tf.nn.relu(tf.nn.conv2d(l3_conv_2,w3_3_2, strides=[1,1,1,1], padding= 'SAME')+b3_3_2)#(?,30, 41, 256)
l3_conv_3=tf.concat([l3_3_1, l3_3_2],3)#(?,30, 41, 512)
l3=tf.nn.max_pool(l3_conv_2, ksize=[1,3,3,1], strides=[1,3,3,1], padding='SAME')#(?,30, 41,512)

#############4 conv layer
l3_4_1=tf.nn.relu(tf.nn.conv2d(l3,w3_4_1, strides=[1,1,1,1], padding= 'SAME')+b3_4_1)#(?,30, 41, 128)
l3_4_2=tf.nn.relu(tf.nn.conv2d(l3,w3_4_2, strides=[1,1,1,1], padding= 'SAME')+b3_4_2)#(?,30, 41, 128)
l3_1=tf.concat([l3_4_1, l3_4_2],3)#(?,30, 41, 256)
l3_2=tf.image.resize_images(l3_1,[int(math.ceil(h/9)),int(math.ceil(w/9))])#(?,10, 12, 256)
l3_3=tf.nn.relu(tf.nn.conv2d_transpose(l3_2,w3_4_3,tf.stack([tf.shape(x)[0],int(math.ceil(h/9)),int(math.ceil(w/9)),256]),strides=[1,1,1,1],padding='SAME'))

#############1 deconv layer
l4_1=tf.image.resize_images(l3_3,[int(math.ceil(h/3)),int(math.ceil(w/3))])#(?,89, 122, 256)
l4_1_1=tf.nn.relu(tf.nn.conv2d(l4_1,w4_1_1, strides=[1,1,1,1], padding= 'SAME')+b4_1_1)#(?,89, 122, 64)
l4_1_2=tf.nn.relu(tf.nn.conv2d(l4_1,w4_1_2, strides=[1,1,1,1], padding= 'SAME')+b4_1_2)#(?,89, 122, 64)
l4_conv_1=tf.concat([l4_1_1, l4_1_2],3)#(?,89, 122, 128)
l4_2_1=tf.nn.relu(tf.nn.conv2d(l4_conv_1,w4_2_1, strides=[1,1,1,1], padding= 'SAME')+b4_2_1)#(?,89, 122, 64)
l4_2_2=tf.nn.relu(tf.nn.conv2d(l4_conv_1,w4_2_2, strides=[1,1,1,1], padding= 'SAME')+b4_2_2)#(?,89, 122, 64)
l4=tf.concat([l4_2_1, l4_2_2],3)#(?,89, 122, 128)

#############2 deconv layer
l5_1=tf.image.resize_images(l4,[h,w])#(?,265,365, 128)
l5_1_1=tf.nn.relu(tf.nn.conv2d(l5_1,w5_1_1, strides=[1,1,1,1], padding= 'SAME')+b5_1_1)#(?,265,365, 32)
l5_1_2=tf.nn.relu(tf.nn.conv2d(l5_1,w5_1_2, strides=[1,1,1,1], padding= 'SAME')+b5_1_2)#(?,265,365, 32)
l5_conv_1=tf.concat([l5_1_1, l5_1_2],3)#(?,265,365, 64)
l5_2_1=tf.nn.relu(tf.nn.conv2d(l5_conv_1,w5_2_1, strides=[1,1,1,1], padding= 'SAME')+b5_2_1)#(?,265,365, 32)
l5_2_2=tf.nn.relu(tf.nn.conv2d(l5_conv_1,w5_2_2, strides=[1,1,1,1], padding= 'SAME')+b5_2_2)#(?,265,365, 32)
l5=tf.concat([l5_2_1, l5_2_2],3)#(?,265,365, 64)


In [84]:
print(l1)
print(l2)
print(l3)
print(l4)
print(l5)
print(l4_1)
print(l3_3)
print(int(math.ceil(h/9)),int(math.ceil(w/9)))

Tensor("MaxPool:0", shape=(?, 71, 98, 64), dtype=float32)
Tensor("MaxPool_1:0", shape=(?, 24, 33, 128), dtype=float32)
Tensor("MaxPool_2:0", shape=(?, 8, 11, 512), dtype=float32)
Tensor("concat_9:0", shape=(?, 71, 98, 128), dtype=float32)
Tensor("concat_10:0", shape=(?, 212, 292, 64), dtype=float32)
Tensor("ResizeBilinear_1:0", shape=(?, 71, 98, ?), dtype=float32)
Tensor("Relu_16:0", shape=(?, ?, ?, ?), dtype=float32)
24 33


In [85]:
w6=tf.get_variable("w6", shape=[1,1,64,128], initializer=tf.contrib.layers.xavier_initializer_conv2d())
b6=tf.Variable(tf.random_normal([128]))
l6=tf.nn.relu(tf.nn.conv2d(l5,w6, strides=[1,1,1,1], padding= 'SAME')+b6)#(?,265,365,512)

w7=tf.get_variable("w7", shape=[1,1,128,38], initializer=tf.contrib.layers.xavier_initializer_conv2d())
b7=tf.Variable(tf.random_normal([38]))
l7=tf.nn.conv2d(l6,w7, strides=[1,1,1,1], padding= 'SAME')+b7#(?,265,365,37)


In [87]:
print(l6)

Tensor("concat_12:0", shape=(?, 212, 292, 64), dtype=float32)


In [88]:
output=tf.reshape(l7,[-1,38])
soft=tf.nn.softmax(output)
flat_y=tf.reshape(y,[-1,38])


entropy=-tf.log(soft)*flat_y
t_cost=tf.reshape(entropy,[-1,h*w*38])
a_loss=tf.reduce_mean(t_cost,1)
cost=tf.reduce_mean(t_cost)

#cost=tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=output,labels=flat_y))



In [89]:
# l10_1=tf.reshape(l10,[-1,265*365,37])
# out=[]

# for i in range(265*365):
#     out.append(tf.nn.softmax(l10_1[:,i,:]))
#     if i%10000 ==0:
#         print(i)
# logits=tf.stack(out)
# logits=tf.transpose(logits,perm=[1,0,2])
# logits=tf.reshape(logits,[-1,265,365,37])
# print(logits)
# cost=tf.log(logits)*y
# print(cost)
# # cost=tf.reduce_mean(tf.reshape(cost,[-1,265*365*37]),1)
# # total_cost=tf.reduce_mean(cost)
# # print(total_cost)
# total_cost=tf.reduce_mean(cost)

In [91]:
learning_rate=tf.train.exponential_decay(0.001,batch_size*batch_size,training_epochs,0.9)
optimizer=tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)
image=tf.argmax(output,3)
y_image=tf.argmax(y,3)


In [ ]:
print("Learning start")
sess=tf.Session()
sess.run(tf.global_variables_initializer())


for epoch in range(training_epochs):
    a=0
    total_batch = int(math.ceil(len(train_image)/batch_size))
    print("epoch : ",epoch)
    for i in range(total_batch):
        if a+batch_size>len(train_image):
            batch_xs=train_image[a:]
            batch_ys=train_label[a:]
        else:
            batch_xs=train_image[a:a+batch_size]
            batch_ys=train_label[a:a+batch_size]
        feed_dict={x:batch_xs, y:batch_ys}
        y_img,img,l,_=sess.run([y_image,image,cost,optimizer], feed_dict=feed_dict)
        if l==tf.is_nan:
            break
        for i in range(len(batch_xs)):
            im=np.minimum(img[i,:,:]*5,255)
            cv2.imwrite('output/output_img-'+str(a+i+1).zfill(6)+'.jpg',im)

        a=a+batch_size
    
    print("loss : ", l)
    plt.figure(figsize=(15,15))
    y_im=np.minimum(y_img[0,:,:]*5,255)
    im1=np.minimum(img[0,:,:]*5,255)
    plt.subplot(1,2,1); plt.imshow(y_im)       
    plt.subplot(1,2,2); plt.imshow(im1)
    plt.show()

Learning start
epoch :  0


In [ ]:
test_img=sess.run([image],feed_dict={x:test_image, y:new_test_label})


In [1]:
cor_pre=tf.equal(tf.arg_max(output,1),tf.arg_max(y,1))
acc_=tf.reduce_mean(tf.cast(cor_pre,tf.float32))
feed_dict={x1:test_image, y:test_label, keep_conv:0.8, keep_hidden:0.5}
test_acc=sess.run(acc_,feed_dict=feed_dict)
print("accuracy : ", test_acc)

NameError: name 'tf' is not defined